https://web.stanford.edu/~jurafsky/slp3/14.pdf

14.1.2 Document Scoring



In [68]:
query = 'sweet love'
doc_1 = 'sweet sweet nurse love'
doc_2 = 'sweet sorrow'
doc_3 = 'how sweet is love'
doc_4 = 'nurse'
docs = [doc_1, doc_2, doc_3, doc_4]

In [69]:
df = {}
for doc in docs:
    word_set = set(doc.split(' '))
    for word in word_set:
        if word in df.keys():
            df[word] += 1
            pass
        else:
            df[word] = 1
            pass
        pass
    pass
df 

{'nurse': 2, 'sweet': 3, 'love': 2, 'sorrow': 1, 'how': 1, 'is': 1}

In [70]:
import pandas as pd
import math

In [81]:
# query table

query_list = query.split(' ')

word = []
cnt = []
tf = []
df_ = []
idf = []
tf_idf = []

for key, value in df.items():
    word.append(key)
    df_.append(value)
    df_t = sum(1 for y in query_list if key == y)
    cnt.append(df_t)
    tf_ = 1 + math.log10(df_t) if df_t > 0 else 0
    tf.append(tf_)    
    idf_ = math.log10(len(docs) / value)
    idf.append(idf_)
    tf_idf_ = tf_ * idf_
    tf_idf.append(tf_idf_)
    pass

data = {
    'word' : word,
    'cnt' : cnt,
    'tf' : tf,
    'df': df_,
    'idf' : idf,
    'tf_idf': tf_idf
}
query_df = pd.DataFrame(data)
abs_q = math.sqrt(query_df['tf_idf'].pow(2).sum()) # |q|
query_df['n\'lized'] = query_df['tf_idf'] / abs_q
query_df

,word,cnt,tf,df,idf,tf_idf,n'lized
0,nurse,0,0.0,2,0.301030,0.000000,0.000000
1,sweet,1,1.0,3,0.124939,0.124939,0.383333
2,love,1,1.0,2,0.301030,0.301030,0.923610
3,sorrow,0,0.0,1,0.602060,0.000000,0.000000
4,how,0,0.0,1,0.602060,0.000000,0.000000
5,is,0,0.0,1,0.602060,0.000000,0.000000


In [91]:
def calculate(doc: str, q_df: pd.DataFrame) -> (pd.DataFrame, float):
    word = []
    cnt = []
    tf = []
    tf_idf = []
    doc_list = doc.split(' ')

    for key, value in df.items():
        word.append(key)
        df_t = sum(1 for y in doc_list if key == y)
        cnt.append(df_t)
        tf_ = 1 + math.log10(df_t) if df_t > 0 else 0
        tf.append(tf_) 
        idf_ = math.log10(len(docs) / value)    
        tf_idf_ = tf_ * idf_
        tf_idf.append(tf_idf_)
        pass

    data = {
        'word' : word,
        'cnt' : cnt,
        'tf' : tf,
        'tf-idf': tf_idf
    }
    doc_df = pd.DataFrame(data)

    abs_d_1 = math.sqrt(doc_df['tf-idf'].pow(2).sum()) # |d_1|
    doc_df['n\'lized'] = doc_df['tf-idf'] / abs_d_1
    doc_df['x q'] = doc_df['n\'lized'] * q_df['n\'lized']

    cosine = doc_df['x q'].sum()

    return doc_df, cosine


In [93]:
print("Document 1")
print(calculate(doc_1, query_df))

Document 1
(     word  cnt       tf    tf-idf   n'lized       x q
0   nurse    1  1.00000  0.301030  0.660592  0.000000
1   sweet    2  1.30103  0.162549  0.356704  0.136736
2    love    1  1.00000  0.301030  0.660592  0.610129
3  sorrow    0  0.00000  0.000000  0.000000  0.000000
4     how    0  0.00000  0.000000  0.000000  0.000000
5      is    0  0.00000  0.000000  0.000000  0.000000, 0.7468654211522301)


In [94]:
print("Document 2")
print(calculate(doc_2, query_df))

Document 2
(     word  cnt   tf    tf-idf   n'lized       x q
0   nurse    0  0.0  0.000000  0.000000  0.000000
1   sweet    1  1.0  0.124939  0.203190  0.077889
2    love    0  0.0  0.000000  0.000000  0.000000
3  sorrow    1  1.0  0.602060  0.979139  0.000000
4     how    0  0.0  0.000000  0.000000  0.000000
5      is    0  0.0  0.000000  0.000000  0.000000, 0.07788932485528884)


In [96]:
print("Document 3")
print(calculate(doc_3, query_df))

print("Document 4")
print(calculate(doc_4, query_df))

Document 3
(     word  cnt   tf    tf-idf   n'lized       x q
0   nurse    0  0.0  0.000000  0.000000  0.000000
1   sweet    1  1.0  0.124939  0.137041  0.052532
2    love    1  1.0  0.301030  0.330188  0.304965
3  sorrow    0  0.0  0.000000  0.000000  0.000000
4     how    1  1.0  0.602060  0.660377  0.000000
5      is    1  1.0  0.602060  0.660377  0.000000, 0.35749763139121166)
Document 4
(     word  cnt   tf   tf-idf  n'lized  x q
0   nurse    1  1.0  0.30103      1.0  0.0
1   sweet    0  0.0  0.00000      0.0  0.0
2    love    0  0.0  0.00000      0.0  0.0
3  sorrow    0  0.0  0.00000      0.0  0.0
4     how    0  0.0  0.00000      0.0  0.0
5      is    0  0.0  0.00000      0.0  0.0, 0.0)
